In [2]:
%load_ext autoreload
%autoreload 2

In [11]:
from cts_recommender.io import readers
from cts_recommender.settings import get_settings
from cts_recommender.features.whatson_csv_schema import ORIGINAL_WHATSON_COLUMNS
cfg = get_settings()

whatson_df = readers.read_csv(cfg.raw_dir /"original_raw_whatson.csv", sep='\t')

In [12]:
assert whatson_df.columns.tolist() == ORIGINAL_WHATSON_COLUMNS